#                                         ELECTRICITY FORECAST
### Comparative Time Series Analysis of the Classic and Econometric Approaches on Production, Consumption and Import of Electricity by Countries

This is an individual project submitted as a written assessment by Ismail Muallaoglu

September 2021

Hyper Island

**Assessment Course Goals to be achieved:**

**Skill 7 - (S7)**

Communicate problems, recommendations and insights adapted to the intended target audience

**Skill 10 - (S10)**

Plan, execute and identify resources for carrying out experiments to draw data-informed conclusions

**Competence 1 - (C1)**

Process and analyze data in such a way that it leads to further learning and professional development

**Competence 3 - (C3)**

Independently be able to work methodically and flexibly in various projects and processes

## TABLE OF CONTENT

* **Project Background**


* **Data Overview**


* **Analysis**

  1) Data Preprocessing
  
  2) Country Selection
   
  3) Methodologies and Functions
       
  4) Visualization
  

* **Conclusion**

## PROJECT BACKGROUND (S7, S10, C3)

**Introduction: (S7)**

Thinking about its wide usage areas, electricity is perhaps the most life-changing innovation. Both domestic and industrial needs of electricity are compelling countries to manage this energy source at a competitive level. To do so, energy data must be analyzed in a higher Business Intelligence Level: Predictive Analysis.

**Methodologie: (S10, C3)**

Being able to forecast production, consumption and import numbers with minimized margin of error, would definitely help us to achieve this purpose. But as time series, the data might have some seasonality and moreover a trend that makes the data non-stationary. Considering that I would like to create a comparative forecast for countries’ data which has such conditions; I am going to use “Holt-Winters: Triple Exponential Smoothing” method as the Classical Approach, “Seasonal Autoregressive Integrated Moving Average” method as the Econometric Approach, and calculate the “Symmetric Mean Absolute Percentage Error” for both methods to compare.

## DATA OVERVIEW (C1)

*Data source: EUROSTAT - Energy monthly data*

**Loading "pandas" and "datetime" libraries**

In [1]:
import pandas as pd
import datetime

**Loading dataset, saving it as "dataset"**

In [2]:
dataset = pd.read_csv('dataset.csv')
dataset

,TIME,GEO,S_ADJ,INDIC,Value,Flag and Footnotes
0,2008M01,European Union - 28 countries (2013-2020),Unadjusted data (i.e. neither seasonally adjus...,"Production of natural gas, TJ (GCV)",:,NaN
1,2008M01,European Union - 28 countries (2013-2020),Unadjusted data (i.e. neither seasonally adjus...,"Total consumption of natural gas, TJ (GCV)",:,NaN
2,2008M01,European Union - 28 countries (2013-2020),Unadjusted data (i.e. neither seasonally adjus...,"Imports of natural gas, TJ (GCV)",:,NaN
3,2008M01,European Union - 28 countries (2013-2020),Unadjusted data (i.e. neither seasonally adjus...,Production of electricity - GWh,:,NaN
4,2008M01,European Union - 28 countries (2013-2020),Unadjusted data (i.e. neither seasonally adjus...,Consumption of electricity - GWh,:,NaN
...,...,...,...,...,...,...
83911,2021M06,Bosnia and Herzegovina,Unadjusted data (i.e. neither seasonally adjus...,Inland deliveries of gas/diesel oil - 1000t,:,NaN
83912,2021M06,Bosnia and Herzegovina,Unadjusted data (i.e. neither seasonally adjus...,Inland deliveries of brown coal - 1000t,:,NaN
83913,2021M06,Bosnia and Herzegovina,Unadjusted data (i.e. neither seasonally adjus...,Imports of crude oil - 1000t,:,NaN
83914,2021M06,Bosnia and Herzegovina,Unadjusted data (i.e. neither seasonally adjus...,Inland deliveries of motor gasoline - 1000t,:,NaN


In [3]:
dataset.shape

(83916, 6)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83916 entries, 0 to 83915
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   TIME                83916 non-null  object
 1   GEO                 83916 non-null  object
 2   S_ADJ               83916 non-null  object
 3   INDIC               83916 non-null  object
 4   Value               83916 non-null  object
 5   Flag and Footnotes  1080 non-null   object
dtypes: object(6)
memory usage: 3.8+ MB


In [5]:
dataset.isnull().sum()

TIME                      0
GEO                       0
S_ADJ                     0
INDIC                     0
Value                     0
Flag and Footnotes    82836
dtype: int64

The data contains:

* Each Month - Year since the first month of 2008 till June 2021.

* Plenty locations

* Different indices of different energy sources

* All columns' data types are objects and there's no null values in Value column. But there are some missing values specified with ":"

## ANALYSIS

### 1) Data Preprocessing (C1, C3)

#### 1.1) Cleaning the data (C1)

**Chosen indices:**

Production of electricity - GWh

Consumption of electricity - GWh

Imports of electricity - GWh

In [6]:
dataset = dataset[(dataset.INDIC == "Production of electricity - GWh") |
                  (dataset.INDIC == "Consumption of electricity - GWh") |
                  (dataset.INDIC == "Imports of electricity - GWh") 
                 ]                          

**Chosing 36 countries except EU total**

In [7]:
dataset = dataset[(dataset.GEO == "Belgium") |
                  (dataset.GEO == "Bulgaria") |
                  (dataset.GEO == "Czechia") |
                  (dataset.GEO == "Denmark") |
                  (dataset.GEO == "Germany (until 1990 former territory of the FRG)") |
                  (dataset.GEO == "Estonia") |
                  (dataset.GEO == "Ireland") |
                  (dataset.GEO == "Greece") |
                  (dataset.GEO == "Spain") |
                  (dataset.GEO == "France") |
                  (dataset.GEO == "Croatia") |
                  (dataset.GEO == "Italy") |
                  (dataset.GEO == "Cyprus") |
                  (dataset.GEO == "Latvia") |
                  (dataset.GEO == "Lithuania") |
                  (dataset.GEO == "Hungary") |
                  (dataset.GEO == "Malta") |
                  (dataset.GEO == "Netherlands") |
                  (dataset.GEO == "Austria") |
                  (dataset.GEO == "Poland") |
                  (dataset.GEO == "Portugal") |
                  (dataset.GEO == "Romania") |
                  (dataset.GEO == "Slovenia") |
                  (dataset.GEO == "Slovakia") |
                  (dataset.GEO == "Finland") |
                  (dataset.GEO == "Sweden") |
                  (dataset.GEO == "Iceland") |
                  (dataset.GEO == "Norway") |
                  (dataset.GEO == "United Kingdom") |
                  (dataset.GEO == "Montenegro") |
                  (dataset.GEO == "North Macedonia") |
                  (dataset.GEO == "Albania") |
                  (dataset.GEO == "Serbia") |
                  (dataset.GEO == "Turkey") |
                  (dataset.GEO == "Belgium") |
                  (dataset.GEO == "Bosnia and Herzegovina")
                 ]              

**Changing the Values that are ":" with 0** 

In [8]:
dataset['Value'] = dataset['Value'].replace([':'], 0)

**Dropping the "S_ADJ", "Flag and Footnotes" columns**

In [9]:
dataset.drop(["S_ADJ", "Flag and Footnotes"], axis=1, inplace=True)

**Renaming Columns**

In [10]:
dataset.rename(columns = {'TIME':'year_month', 'GEO':'country', 'Value':'value'}, inplace=True)

**String parsing in value column**

In [11]:
dataset.value = dataset['value'].replace('[\ ]', '', regex=True) #getting rid of spaces 
dataset.value = pd.to_numeric(dataset.value) #convert strings containing numerical values into floats
dataset.value = dataset['value'].round(decimals=0)

**String parsing in year_month column**

In [12]:
dataset["year_month"] = dataset["year_month"].str.replace("M", "-") #changing M with -
dataset['year_month'] = pd.to_datetime(dataset['year_month']) #changing the format from object to datetime

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17010 entries, 17 to 83907
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year_month  17010 non-null  datetime64[ns]
 1   country     17010 non-null  object        
 2   INDIC       17010 non-null  object        
 3   value       17010 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 664.5+ KB


**Checking if there are any null values**

In [14]:
dataset.isnull().sum()

year_month    0
country       0
INDIC         0
value         0
dtype: int64

In [15]:
dataset.describe()

,value
count,17010.000000
mean,5136.023751
std,8965.335053
min,-75.000000
25%,361.000000
50%,1365.000000
75%,5141.750000
max,61424.000000


**Resetting index**

In [16]:
dataset.reset_index(inplace=True)

**When we reset index, old index became an additional column, we need to drop this**

In [17]:
dataset.drop(["index"], axis=1, inplace=True)

**Changing Germany's name to avoid of confusions**

In [18]:
dataset = dataset.set_index('country')
dataset = dataset.rename(index={'Germany (until 1990 former territory of the FRG)':'Germany'})

**Checking the outlook of dataframe**

In [19]:
dataset.head()

,year_month,INDIC,value
country,,,
Belgium,2008-01-01,Production of electricity - GWh,7709.0
Belgium,2008-01-01,Consumption of electricity - GWh,8849.0
Belgium,2008-01-01,Imports of electricity - GWh,1588.0
Bulgaria,2008-01-01,Production of electricity - GWh,4434.0
Bulgaria,2008-01-01,Consumption of electricity - GWh,4327.0


#### 1.2) Creating Production, Consumption and Import files (C3)

I will create seperate files for production, consumption and import of electricity; then I will go on with only the countries which have full data.

##### Electricity Production

In [20]:
production = dataset[dataset.INDIC == 'Production of electricity - GWh']

In [21]:
# Resetting index

production.reset_index(inplace=True)
production.head()

,country,year_month,INDIC,value
0,Belgium,2008-01-01,Production of electricity - GWh,7709.0
1,Bulgaria,2008-01-01,Production of electricity - GWh,4434.0
2,Czechia,2008-01-01,Production of electricity - GWh,8499.0
3,Denmark,2008-01-01,Production of electricity - GWh,4027.0
4,Germany,2008-01-01,Production of electricity - GWh,0.0


**Creating pivot table**

year_month on rows

country on columns

In [22]:
production = production.pivot(index='year_month', columns='country', values='value')
production

country,Albania,Austria,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,...,Poland,Portugal,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom
year_month,,,,,,,,,,,,,,,,,,,,,
2008-01-01,0.0,5528.0,7709.0,0.0,4434.0,1104.0,461.0,8499.0,4027.0,1125.0,...,14755.0,3259.0,6218.0,0.0,2791.0,1361.0,27405.0,15424.0,17888.0,38520.0
2008-02-01,0.0,5097.0,7124.0,0.0,3678.0,1028.0,415.0,7769.0,3494.0,996.0,...,13339.0,2658.0,5841.0,0.0,2609.0,1269.0,25549.0,14366.0,16571.0,34973.0
2008-03-01,0.0,5561.0,6716.0,0.0,3780.0,1050.0,340.0,7830.0,3225.0,762.0,...,13269.0,2279.0,5752.0,0.0,2708.0,1316.0,26248.0,14990.0,16345.0,35599.0
2008-04-01,0.0,5319.0,6330.0,0.0,3559.0,1065.0,323.0,7391.0,2601.0,778.0,...,12478.0,2106.0,5168.0,0.0,2235.0,1395.0,24813.0,13568.0,15710.0,33138.0
2008-05-01,0.0,5676.0,6481.0,0.0,3381.0,929.0,358.0,6393.0,2337.0,675.0,...,11846.0,2451.0,5009.0,0.0,2637.0,1379.0,23847.0,12562.0,16215.0,29040.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01,1187.0,5240.0,8119.0,1850.0,3451.0,1541.0,338.0,6780.0,3330.0,574.0,...,12813.0,5309.0,4697.0,3633.0,2498.0,1284.0,21112.0,14819.0,22930.0,0.0
2021-03-01,1017.0,5126.0,8966.0,1602.0,3918.0,1401.0,348.0,7026.0,3070.0,394.0,...,13280.0,4495.0,4953.0,3674.0,2458.0,1312.0,22057.0,15837.0,26720.0,0.0
2021-04-01,1004.0,5126.0,8038.0,1220.0,3530.0,1083.0,299.0,6034.0,2790.0,415.0,...,12618.0,3939.0,4453.0,3077.0,2153.0,735.0,19882.0,14759.0,24847.0,0.0


In [23]:
# Converting the pivot table into a df

production.columns.name = None    #remove year_month
production = production.reset_index() #index to columns
production.head()

,year_month,Albania,Austria,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,Czechia,Denmark,...,Poland,Portugal,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom
0,2008-01-01,0.0,5528.0,7709.0,0.0,4434.0,1104.0,461.0,8499.0,4027.0,...,14755.0,3259.0,6218.0,0.0,2791.0,1361.0,27405.0,15424.0,17888.0,38520.0
1,2008-02-01,0.0,5097.0,7124.0,0.0,3678.0,1028.0,415.0,7769.0,3494.0,...,13339.0,2658.0,5841.0,0.0,2609.0,1269.0,25549.0,14366.0,16571.0,34973.0
2,2008-03-01,0.0,5561.0,6716.0,0.0,3780.0,1050.0,340.0,7830.0,3225.0,...,13269.0,2279.0,5752.0,0.0,2708.0,1316.0,26248.0,14990.0,16345.0,35599.0
3,2008-04-01,0.0,5319.0,6330.0,0.0,3559.0,1065.0,323.0,7391.0,2601.0,...,12478.0,2106.0,5168.0,0.0,2235.0,1395.0,24813.0,13568.0,15710.0,33138.0
4,2008-05-01,0.0,5676.0,6481.0,0.0,3381.0,929.0,358.0,6393.0,2337.0,...,11846.0,2451.0,5009.0,0.0,2637.0,1379.0,23847.0,12562.0,16215.0,29040.0


**Save this df as another xlsx file**

In [24]:
production.to_excel('C:/Users/ismai/Desktop/electricity_forecast/electricity_data/production/production.xlsx', index=False)

**Which countries have full production data?**

In [25]:
production.apply(lambda s : s.value_counts().get(key=0,default=0), axis=0)  

year_month                  1
Albania                   120
Austria                     0
Belgium                     1
Bosnia and Herzegovina    119
Bulgaria                    1
Croatia                     0
Cyprus                      0
Czechia                     1
Denmark                     0
Estonia                     1
Finland                     1
France                     21
Germany                    97
Greece                      0
Hungary                     1
Iceland                    77
Ireland                     0
Italy                       1
Latvia                      0
Lithuania                   1
Malta                      10
Montenegro                 86
Netherlands                 1
North Macedonia            97
Norway                      1
Poland                      1
Portugal                    1
Romania                     1
Serbia                     97
Slovakia                    1
Slovenia                    1
Spain                       0
Sweden    

**Apparently we need to choose:**\
Austria\
Croatia\
Cyprus\
Denmark\
Greece\
Ireland\
Latvia\
Spain\
Sweden

##### Electricity Consumption

In [26]:
consumption = dataset[dataset.INDIC == 'Consumption of electricity - GWh']

In [27]:
# Resetting index

consumption.reset_index(inplace=True)
consumption.head()

,country,year_month,INDIC,value
0,Belgium,2008-01-01,Consumption of electricity - GWh,8849.0
1,Bulgaria,2008-01-01,Consumption of electricity - GWh,4327.0
2,Czechia,2008-01-01,Consumption of electricity - GWh,7031.0
3,Denmark,2008-01-01,Consumption of electricity - GWh,3594.0
4,Germany,2008-01-01,Consumption of electricity - GWh,0.0


**Creating pivot table**\
year_month on columns\
country on rows

In [28]:
consumption = consumption.pivot(index='year_month', columns='country', values='value')
consumption

country,Albania,Austria,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,...,Poland,Portugal,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom
year_month,,,,,,,,,,,,,,,,,,,,,
2008-01-01,0.0,6637.0,8849.0,0.0,4327.0,1663.0,461.0,7031.0,3594.0,975.0,...,14566.0,3973.0,5730.0,0.0,2735.0,1322.0,26944.0,14527.0,17947.0,39452.0
2008-02-01,0.0,6085.0,8349.0,0.0,3682.0,1564.0,415.0,6485.0,3293.0,887.0,...,13208.0,3434.0,5290.0,0.0,2490.0,1247.0,24885.0,13464.0,16610.0,35781.0
2008-03-01,0.0,6259.0,8476.0,0.0,3352.0,1591.0,340.0,6643.0,3327.0,887.0,...,13335.0,3320.0,5286.0,0.0,2564.0,1289.0,25037.0,13973.0,16215.0,36316.0
2008-04-01,0.0,5777.0,7927.0,0.0,2974.0,1480.0,323.0,6092.0,3073.0,749.0,...,12487.0,3004.0,4812.0,0.0,2323.0,1215.0,23519.0,12056.0,15653.0,34085.0
2008-05-01,0.0,5699.0,7635.0,0.0,2794.0,1428.0,358.0,5621.0,2957.0,704.0,...,11842.0,3225.0,4728.0,0.0,2356.0,1192.0,23087.0,10916.0,16260.0,30437.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01,598.0,5417.0,6761.0,924.0,2804.0,1345.0,338.0,5658.0,3224.0,852.0,...,12864.0,3816.0,4769.0,3072.0,2334.0,1034.0,18245.0,12599.0,22790.0,0.0
2021-03-01,617.0,5838.0,7050.0,1025.0,3029.0,1446.0,348.0,5851.0,3168.0,852.0,...,13624.0,3870.0,5069.0,3315.0,2485.0,1148.0,19698.0,12416.0,26444.0,0.0
2021-04-01,594.0,5292.0,6548.0,919.0,2624.0,1332.0,299.0,5230.0,2831.0,723.0,...,12554.0,3585.0,4606.0,2974.0,2255.0,1043.0,17984.0,10879.0,24634.0,0.0


In [29]:
# Converting the pivot table into a df

consumption.columns.name = None    #remove year_month
consumption = consumption.reset_index() #index to columns
consumption.head()

,year_month,Albania,Austria,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,Czechia,Denmark,...,Poland,Portugal,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom
0,2008-01-01,0.0,6637.0,8849.0,0.0,4327.0,1663.0,461.0,7031.0,3594.0,...,14566.0,3973.0,5730.0,0.0,2735.0,1322.0,26944.0,14527.0,17947.0,39452.0
1,2008-02-01,0.0,6085.0,8349.0,0.0,3682.0,1564.0,415.0,6485.0,3293.0,...,13208.0,3434.0,5290.0,0.0,2490.0,1247.0,24885.0,13464.0,16610.0,35781.0
2,2008-03-01,0.0,6259.0,8476.0,0.0,3352.0,1591.0,340.0,6643.0,3327.0,...,13335.0,3320.0,5286.0,0.0,2564.0,1289.0,25037.0,13973.0,16215.0,36316.0
3,2008-04-01,0.0,5777.0,7927.0,0.0,2974.0,1480.0,323.0,6092.0,3073.0,...,12487.0,3004.0,4812.0,0.0,2323.0,1215.0,23519.0,12056.0,15653.0,34085.0
4,2008-05-01,0.0,5699.0,7635.0,0.0,2794.0,1428.0,358.0,5621.0,2957.0,...,11842.0,3225.0,4728.0,0.0,2356.0,1192.0,23087.0,10916.0,16260.0,30437.0


**Save this df as another xlsx file**

In [30]:
consumption.to_excel('C:/Users/ismai/Desktop/electricity_forecast/electricity_data/consumption/consumption.xlsx', index=False)

**Which countries have full consumption data?**

In [31]:
consumption.apply(lambda s : s.value_counts().get(key=0,default=0), axis=0)  

year_month                  1
Albania                   120
Austria                     0
Belgium                     1
Bosnia and Herzegovina    119
Bulgaria                    1
Croatia                     0
Cyprus                      0
Czechia                     1
Denmark                     0
Estonia                     1
Finland                     1
France                     21
Germany                    97
Greece                      0
Hungary                     1
Iceland                    77
Ireland                     0
Italy                       1
Latvia                      0
Lithuania                   1
Malta                      10
Montenegro                 86
Netherlands                 1
North Macedonia            97
Norway                      1
Poland                      1
Portugal                    1
Romania                     1
Serbia                     97
Slovakia                    1
Slovenia                    1
Spain                       0
Sweden    

**Apparently we need to choose:**\
Austria\
Croatia\
Cyprus\
Denmark\
Greece\
Ireland\
Latvia\
Spain\
Sweden

##### Electricity Import

In [32]:
imp = dataset[dataset.INDIC == 'Imports of electricity - GWh']

In [33]:
# Resetting index

imp.reset_index(inplace=True)
imp.head()

,country,year_month,INDIC,value
0,Belgium,2008-01-01,Imports of electricity - GWh,1588.0
1,Bulgaria,2008-01-01,Imports of electricity - GWh,467.0
2,Czechia,2008-01-01,Imports of electricity - GWh,1065.0
3,Denmark,2008-01-01,Imports of electricity - GWh,752.0
4,Germany,2008-01-01,Imports of electricity - GWh,0.0


**Creating pivot table**\
year_month on columns\
country on rows

In [34]:
imp = imp.pivot(index='year_month', columns='country', values='value')
imp

country,Albania,Austria,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,...,Poland,Portugal,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom
year_month,,,,,,,,,,,,,,,,,,,,,
2008-01-01,0.0,2503.0,1588.0,0.0,467.0,724.0,0.0,1065.0,752.0,17.0,...,1008.0,894.0,90.0,0.0,1153.0,587.0,691.0,840.0,65.0,1021.0
2008-02-01,0.0,2284.0,1569.0,0.0,544.0,694.0,0.0,865.0,862.0,10.0,...,962.0,858.0,65.0,0.0,995.0,553.0,608.0,894.0,66.0,902.0
2008-03-01,0.0,2162.0,2131.0,0.0,318.0,690.0,0.0,708.0,897.0,220.0,...,950.0,1079.0,60.0,0.0,724.0,674.0,230.0,933.0,46.0,791.0
2008-04-01,0.0,1764.0,1886.0,0.0,141.0,628.0,0.0,569.0,1366.0,164.0,...,699.0,959.0,45.0,0.0,736.0,575.0,336.0,379.0,68.0,1020.0
2008-05-01,0.0,1287.0,1603.0,0.0,118.0,660.0,0.0,507.0,1279.0,153.0,...,583.0,854.0,58.0,0.0,431.0,456.0,580.0,351.0,125.0,1437.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01,15.0,2094.0,1112.0,173.0,196.0,943.0,0.0,866.0,1270.0,468.0,...,1344.0,258.0,668.0,311.0,639.0,665.0,1350.0,1247.0,84.0,0.0
2021-03-01,42.0,2568.0,905.0,260.0,213.0,900.0,0.0,1160.0,1776.0,659.0,...,1849.0,579.0,684.0,401.0,882.0,617.0,1537.0,579.0,91.0,0.0
2021-04-01,45.0,2346.0,1107.0,320.0,121.0,924.0,0.0,1450.0,1616.0,645.0,...,1507.0,675.0,758.0,497.0,1110.0,925.0,1500.0,344.0,89.0,0.0


In [35]:
# Converting the pivot table into a df

imp.columns.name = None    #remove year_month
imp = imp.reset_index() #index to columns
imp.head()

,year_month,Albania,Austria,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,Czechia,Denmark,...,Poland,Portugal,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom
0,2008-01-01,0.0,2503.0,1588.0,0.0,467.0,724.0,0.0,1065.0,752.0,...,1008.0,894.0,90.0,0.0,1153.0,587.0,691.0,840.0,65.0,1021.0
1,2008-02-01,0.0,2284.0,1569.0,0.0,544.0,694.0,0.0,865.0,862.0,...,962.0,858.0,65.0,0.0,995.0,553.0,608.0,894.0,66.0,902.0
2,2008-03-01,0.0,2162.0,2131.0,0.0,318.0,690.0,0.0,708.0,897.0,...,950.0,1079.0,60.0,0.0,724.0,674.0,230.0,933.0,46.0,791.0
3,2008-04-01,0.0,1764.0,1886.0,0.0,141.0,628.0,0.0,569.0,1366.0,...,699.0,959.0,45.0,0.0,736.0,575.0,336.0,379.0,68.0,1020.0
4,2008-05-01,0.0,1287.0,1603.0,0.0,118.0,660.0,0.0,507.0,1279.0,...,583.0,854.0,58.0,0.0,431.0,456.0,580.0,351.0,125.0,1437.0


**Save this df as another xlsx file**

In [36]:
imp.to_excel('C:/Users/ismai/Desktop/electricity_forecast/electricity_data/import/imp.xlsx', index=False)

**Which countries have full import data?**

In [37]:
imp.apply(lambda s : s.value_counts().get(key=0,default=0), axis=0)    

year_month                  1
Albania                   120
Austria                     0
Belgium                     1
Bosnia and Herzegovina    119
Bulgaria                    1
Croatia                     0
Cyprus                    162
Czechia                     1
Denmark                     0
Estonia                     1
Finland                     1
France                     21
Germany                    97
Greece                      0
Hungary                     1
Iceland                   162
Ireland                     0
Italy                       1
Latvia                      0
Lithuania                   1
Malta                      88
Montenegro                 86
Netherlands                 2
North Macedonia            97
Norway                      1
Poland                      1
Portugal                    1
Romania                     1
Serbia                     97
Slovakia                    1
Slovenia                    1
Spain                       0
Sweden    

**Apparently we need to choose:**\
Austria\
Croatia\
Denmark\
Greece\
Ireland\
Latvia\
Spain\
Sweden

These 8 countries are the ones we are going to test whether they are stationary.